In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

import tensorflow as tf
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow import squeeze

import numpy as np

print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

import configparser
config = configparser.ConfigParser()
config.read('config.ini')

import wandb
from wandb.keras import WandbCallback
wandb.login()

run = wandb.init(project="basic-nn-tensorflow", id="lr-scheduler-adam-bce-cos-deeper")
wandb.config.update(config)



2.9.2
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mato-katkovcin (nsiete-hrnciar-katkovcin). Use `wandb login --relogin` to force relogin


Initial load of config file, where we have hyperparameters we are training our model

In [2]:
start_lr = config['tensorflow'].getfloat('start_lr')
max_lr = config['tensorflow'].getfloat('max_lr')
epochs = config['default'].getint('epochs')
steps_per_epoch = config['tensorflow'].getint('steps_per_epoch')
anneal_strategy = config['tensorflow']['strategy']

start_lr, max_lr, epochs, steps_per_epoch, anneal_strategy

(0.001, 0.01, 500, 1, 'cos')

## Data loading and train-test split

Initially, we imported the processed data and formed two collections of information: predictors (X) and labels (y). We then divided these sets into two parts, one for training (80%) and another for testing (20%). To maintain consistency in the split, we also specified a random state seed.

In [3]:
df = pd.read_csv('Data/cleaned.csv', index_col=0)
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,0.352941,0.670968,0.489796,0.304348,0.186899,0.314928,0.234415,0.483333,1
1,0.058824,0.264516,0.428571,0.239130,0.106370,0.171779,0.116567,0.166667,0
2,0.470588,0.896774,0.408163,0.271739,0.186899,0.104294,0.253629,0.183333,1
3,0.058824,0.290323,0.428571,0.173913,0.096154,0.202454,0.038002,0.000000,0
4,0.000000,0.600000,0.163265,0.304348,0.185096,0.509202,0.943638,0.200000,1
...,...,...,...,...,...,...,...,...,...
763,0.588235,0.367742,0.530612,0.445652,0.199519,0.300613,0.039710,0.700000,0
764,0.117647,0.503226,0.469388,0.217391,0.106370,0.380368,0.111870,0.100000,0
765,0.294118,0.496774,0.489796,0.173913,0.117788,0.163599,0.071307,0.150000,0
766,0.058824,0.529032,0.367347,0.271739,0.186899,0.243354,0.115713,0.433333,1


In [4]:
X, y = df.drop('Outcome', axis=1).values, df.Outcome.values

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=True, random_state=42)

X_train = tf.cast(tf.constant(X_train), tf.float32)
X_test = tf.cast(tf.constant(X_test), tf.float32)
y_train = tf.cast(tf.expand_dims(y_train, axis=1), tf.float32)
y_test = tf.cast(tf.expand_dims(y_test, axis=1), tf.float32)

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-03-29 11:33:10.041799: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-03-29 11:33:10.043446: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


## Neural network

Our neural network is composed of three layers: an input layer, a hidden layer, and an output layer. The activation function used in all layers except the output layer is ReLU, while the output layer uses the Sigmoid activation function, which ensures the output is within the range of 0 to 1. To perform binary classification, we utilized binary cross entropy loss (BCELoss) and Adam optimizer.
Furthermore, we incorporated a learning rate scheduler to adjust the learning rate, making it higher at the start of training and gradually decreasing as we approach the minimum.

### Parameters

- epochs = 500
- base learning rate = 0.001
- max learning rate = 0.01
- annealing strategy = linear
- steps per epoch = 1

### Experiments

Link to wandb report: https://api.wandb.ai/links/nsiete-hrnciar-katkovcin/9v9fld5d

1. **Bigger net, LearningRateScheduler with cos annealing strategy** `lr-scheduler-adam-bce-cos-deeper)` - This refers to a neural network with four layers: an input layer with 8 neurons, two hidden layers with 16 and 8 neurons, respectively, and an output layer with 1 neuron. The ReLU activation function is used on all layers except for the output layer, which uses the Sigmoid activation function. The BCELoss loss function is used with the Adam optimizer and the LearningRateScheduler scheduler with a cosinus annealing strategy. However, this neural network suffers from overfitting, which is indicated by the validation loss diverging from the training loss. This means that while the training loss is decreasing and the training accuracy is increasing, the validation loss is increasing and the validation accuracy is decreasing.


2. **LearningRateScheduler scheduler with _cos_ annealing strategy** `(lr-scheduler-adam-bce)` - This describes a neural network with three layers: an input layer with 8 neurons, a hidden layer with 4 neurons, and an output layer with 1 neuron. The activation function, loss function, and optimizer are the same as the previous network. The scheduler is also the same (LEatningRateScheduler) with cos annealing strategy. Despite being smaller in size, this neural network did not suffer from overfitting. However, its training accuracy was lower at 87% and validation accuracy 84% and its loss was higher compared to the previous network.

3. **LearningRateScheduler scheduler with _linear_ annealing strategy** `(lr-scheduler-adam-bce-linear)` - The network utilized the LearningRateScheduler scheduler with a linear annealing strategy, and it produced slightly improved results compared to using the cos strategy. The training accuracy was 88% and validation accuracy was 86%, and it did not suffer from overfitting, but the loss was higher than in the previous experiment.

4. **No scheduler with learning rate 0.01 with cos annealing strategy** `(lr-0.01-adam-bce)` - The same network was used without the scheduler, resulting in higher loss and the training accuracy was 89% and validation accuracy 80%. This mean there was an overfitting. This could be due to a small learning rate and step, which significantly increased the training time, and the network may have been stuck in a local minimum. Alternatively, it may not have had enough time or epochs to converge to a good enough result.

5. **No scheduler with learning rate 0.001 with cos annealing strategy** `(lr-0.001-adam-bce-cos2)` - The parameters used were the same as in the previous experiment, except for a higher learning rate. The network was able to converge to a better result in this case, with a validation accuracy of 84% and training accuracy at 87%. However, this resulted in overfitting, as indicated by the diverging losses.


In [5]:
def schedule(epoch):
    cycle = 0
    if epoch < (epochs // 2):
        cycle = 1 + (epoch * steps_per_epoch) // (epochs // 2)
        factor = float(cycle * max_lr) / float(epochs)
    else:
        cycle = 1 + ((epochs - epoch) * steps_per_epoch) // (epochs // 2)
        factor = float(cycle * max_lr) / float(epochs)
        factor = max_lr - factor

    if anneal_strategy == 'cos':
        lr = factor * (1 + tf.math.cos(tf.constant(np.pi) * (epoch % (epochs // 2)) / (epochs // 2))) / 2
    elif anneal_strategy == 'linear':
        lr = factor * (epochs - epoch) / epochs
    else:
        raise ValueError('Invalid anneal strategy')

    return lr

In [6]:
def NeuralNetworkModel():
    model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation='relu', input_shape=[len(df.keys()) - 1]),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])
    return model

model = NeuralNetworkModel()

loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
lr_scheduler = LearningRateScheduler(schedule=schedule, verbose=True)

model.compile(loss=loss_fn, optimizer=optimizer, metrics=['accuracy'])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 16)                144       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 361
Trainable params: 361
Non-trainable params: 0
_________________________________________________________________


During the training process, we measure the loss and accuracy while the scheduler adjusts the learning rate. When testing, we measure the testing loss,  and accuracy and all these metrics are recorded and stored in wandb.

In [8]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=config['default'].getint('epochs'), verbose=2, callbacks=[WandbCallback(valdiation_data=(X_test, y_test), log_weights=True)])

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/500


2023-03-29 11:33:10.587793: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
/opt/homebrew/Caskroom/miniconda/base/envs/nsietetensor/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)
2023-03-29 11:33:10.867651: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-03-29 11:33:11.782559: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:Assets written to: /Users/martinkatkovcin/Documents/School/NSIETE/Zadanie1/wandb/run-20230329_113305-lr-scheduler-adam-bce-cos-deeper/files/model-best/assets


wandb: Adding directory to artifact (/Users/martinkatkovcin/Documents/School/NSIETE/Zadanie1/wandb/run-20230329_113305-lr-scheduler-adam-bce-cos-deeper/files/model-best)... Done. 0.0s


...

20/20 - 1s - loss: 0.3174 - accuracy: 0.8583 - val_loss: 0.4238 - val_accuracy: 0.8506 - 646ms/epoch - 32ms/step
Epoch 99/500
20/20 - 0s - loss: 0.3176 - accuracy: 0.8550 - val_loss: 0.4309 - val_accuracy: 0.8506 - 180ms/epoch - 9ms/step
Epoch 100/500
20/20 - 0s - loss: 0.3166 - accuracy: 0.8632 - val_loss: 0.4335 - val_accuracy: 0.8571 - 169ms/epoch - 8ms/step
Epoch 101/500
20/20 - 0s - loss: 0.3168 - accuracy: 0.8567 - val_loss: 0.4365 - val_accuracy: 0.8442 - 176ms/epoch - 9ms/step
Epoch 102/500
20/20 - 0s - loss: 0.3162 - accuracy: 0.8713 - val_loss: 0.4300 - val_accuracy: 0.8571 - 165ms/epoch - 8ms/step
Epoch 103/500
20/20 - 0s - loss: 0.3137 - accuracy: 0.8616 - val_loss: 0.4239 - val_accuracy: 0.8571 - 170ms/epoch - 9ms/step
Epoch 104/500
20/20 - 0s - loss: 0.3152 - accuracy: 0.8583 - val_loss: 0.4446 - val_accuracy: 0.8377 - 169ms/epoch - 8ms/step
Epoch 105/500
20/20 - 0s - loss: 0.3234 - accuracy: 0.8664 - val_loss: 0.4303 - val_accuracy: 0.8636 - 206ms/epoch - 10ms/step
Epoch

wandb: Adding directory to artifact (/Users/martinkatkovcin/Documents/School/NSIETE/Zadanie1/wandb/run-20230329_113305-lr-scheduler-adam-bce-cos-deeper/files/model-best)... Done. 0.0s


20/20 - 1s - loss: 0.3121 - accuracy: 0.8648 - val_loss: 0.4227 - val_accuracy: 0.8701 - 1s/epoch - 67ms/step
Epoch 108/500
20/20 - 0s - loss: 0.3112 - accuracy: 0.8616 - val_loss: 0.4315 - val_accuracy: 0.8571 - 335ms/epoch - 17ms/step
Epoch 109/500
20/20 - 0s - loss: 0.3150 - accuracy: 0.8583 - val_loss: 0.4394 - val_accuracy: 0.8571 - 182ms/epoch - 9ms/step
Epoch 110/500
20/20 - 0s - loss: 0.3133 - accuracy: 0.8713 - val_loss: 0.4239 - val_accuracy: 0.8636 - 165ms/epoch - 8ms/step
Epoch 111/500
20/20 - 0s - loss: 0.3161 - accuracy: 0.8648 - val_loss: 0.4379 - val_accuracy: 0.8506 - 164ms/epoch - 8ms/step
Epoch 112/500
20/20 - 0s - loss: 0.3116 - accuracy: 0.8730 - val_loss: 0.4304 - val_accuracy: 0.8636 - 164ms/epoch - 8ms/step
Epoch 113/500
20/20 - 0s - loss: 0.3124 - accuracy: 0.8681 - val_loss: 0.4268 - val_accuracy: 0.8636 - 159ms/epoch - 8ms/step
Epoch 114/500
20/20 - 0s - loss: 0.3097 - accuracy: 0.8664 - val_loss: 0.4280 - val_accuracy: 0.8571 - 157ms/epoch - 8ms/step
Epoch 1

wandb: Adding directory to artifact (/Users/martinkatkovcin/Documents/School/NSIETE/Zadanie1/wandb/run-20230329_113305-lr-scheduler-adam-bce-cos-deeper/files/model-best)... Done. 0.0s


20/20 - 1s - loss: 0.3061 - accuracy: 0.8681 - val_loss: 0.4215 - val_accuracy: 0.8701 - 666ms/epoch - 33ms/step
Epoch 132/500
20/20 - 0s - loss: 0.3068 - accuracy: 0.8681 - val_loss: 0.4409 - val_accuracy: 0.8636 - 231ms/epoch - 12ms/step
Epoch 133/500
20/20 - 0s - loss: 0.3064 - accuracy: 0.8876 - val_loss: 0.4295 - val_accuracy: 0.8571 - 208ms/epoch - 10ms/step
Epoch 134/500
20/20 - 0s - loss: 0.3051 - accuracy: 0.8730 - val_loss: 0.4412 - val_accuracy: 0.8506 - 174ms/epoch - 9ms/step
Epoch 135/500
20/20 - 0s - loss: 0.3023 - accuracy: 0.8746 - val_loss: 0.4260 - val_accuracy: 0.8701 - 164ms/epoch - 8ms/step
Epoch 136/500
20/20 - 0s - loss: 0.3015 - accuracy: 0.8664 - val_loss: 0.4361 - val_accuracy: 0.8442 - 234ms/epoch - 12ms/step
Epoch 137/500
20/20 - 0s - loss: 0.3013 - accuracy: 0.8762 - val_loss: 0.4334 - val_accuracy: 0.8636 - 163ms/epoch - 8ms/step
Epoch 138/500
20/20 - 0s - loss: 0.3034 - accuracy: 0.8746 - val_loss: 0.4344 - val_accuracy: 0.8442 - 167ms/epoch - 8ms/step
Ep

In [9]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

,loss,accuracy,val_loss,val_accuracy,epoch
495,0.259973,0.884365,0.457235,0.844156,495
496,0.259195,0.887622,0.440586,0.850649,496
497,0.259959,0.895765,0.457168,0.844156,497
498,0.263187,0.881108,0.456230,0.850649,498
499,0.258270,0.890880,0.442118,0.857143,499


At the end, we calculate training and validation accuracy.

In [10]:
acc = (hist['accuracy'].tail().sum())*100/5 
val_acc = (hist['val_accuracy'].tail().sum())*100/5 

print("Training Accuracy = {}% and Validation Accuracy = {}%".format(round(acc, 2), round(val_acc, 2)))

Training Accuracy = 88.79% and Validation Accuracy = 84.94%


In [11]:
#tf.keras.models.save_model(model, "models/model_tf.pth")
wandb.save('runs/pima_run_2023-03-28-tf-adam-bce-cos-lr-deeper')
wandb.finish()

accuracy,▁▅▅▅▆▆▇▇▇▇▇▇████████████████████████▇███
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▅▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▄▅▅▆▆▇█▇██████████▇███████████▇███▇▇▇▇▇
val_loss,█▄▂▂▂▁▁▁▂▁▁▁▁▁▁▂▁▁▁▁▁▂▂▁▁▁▂▁▂▂▁▂▂▂▂▂▂▂▃▂
accuracy,0.89088
best_epoch,130
best_val_loss,0.42148
epoch,499
loss,0.25827
val_accuracy,0.85714
